# Azure AI Context Providers for Agent Memory

This notebook shows how to design and compose context providers when building Azure AI Agents with the Agent Framework.

> Based on the [Agent Memory documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-memory?pivots=programming-language-python).


## Prerequisites

- Azure CLI login (`az login`) with access to the Azure AI Foundry project
- Environment variables configured in `../.env` (`AZURE_AI_PROJECT_ENDPOINT`, `AZURE_AI_MODEL_DEPLOYMENT_NAME`, optional `TENANT_ID`)
- Python packages: `agent-framework`, `agent-framework-azure-ai`, `python-dotenv`, `azure-identity`
- Python 3.10+ kernel for running the notebook


## Imports and configuration

We start by loading environment variables and validating that the Azure AI settings are available. These values are shared with the other Azure AI notebooks in this repository.


In [25]:
import asyncio
import os
import re
from collections import defaultdict
from dataclasses import dataclass, field
from typing import Iterable, MutableSequence, Sequence

from dotenv import load_dotenv

from agent_framework import AggregateContextProvider, ChatAgent, ChatMessage, Context, ContextProvider
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential


In [26]:
# Load environment variables shared across the project
load_dotenv('../.env')

required_vars = [
    'AZURE_AI_PROJECT_ENDPOINT',
    'AZURE_AI_MODEL_DEPLOYMENT_NAME',
]

missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    raise RuntimeError(f'Missing required environment variables: {missing}')

endpoint = os.getenv('AZURE_AI_PROJECT_ENDPOINT')
deployment = os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print('AZURE_AI_PROJECT_ENDPOINT:', endpoint)
print('AZURE_AI_MODEL_DEPLOYMENT_NAME:', deployment)


AZURE_AI_PROJECT_ENDPOINT: https://kd-foundry-project-resource.services.ai.azure.com/api/projects/kd-foundry-project
AZURE_AI_MODEL_DEPLOYMENT_NAME: gpt-4o


## How context providers work

`ContextProvider` instances observe the chat lifecycle:

- `thread_created` triggers when a new thread is initialised.
- `invoking` runs before each Azure AI invocation and can contribute instructions, messages, or tools.
- `invoked` runs after the invocation and can be used to store new information.

Multiple providers can be combined using `AggregateContextProvider`. The Agent Framework merges the resulting `Context` objects and sends them to Azure AI Foundry as part of the request payload.


## In-memory fact store

We use a simple in-memory store so that the notebook stays self-contained. For production workloads replace this with durable storage such as Azure Cosmos DB, Azure Table Storage, or Azure Cache for Redis.


In [27]:
@dataclass
class FactStore:
    facts: defaultdict[str, set[str]] = field(default_factory=lambda: defaultdict(set))

    def load(self, user_id: str) -> list[str]:
        # Return sorted facts for deterministic output.
        return sorted(self.facts[user_id])

    def add(self, user_id: str, new_facts: Iterable[str]) -> None:
        bucket = self.facts[user_id]
        for item in new_facts:
            cleaned = item.strip()
            if cleaned:
                bucket.add(cleaned)

    def clear(self, user_id: str) -> None:
        self.facts.pop(user_id, None)


## User fact context provider

The provider listens to user utterances for preference statements and converts them into memories that will be appended to each request as additional instructions. This mirrors the custom memory providers described in the documentation, but uses Azure AI Foundry for execution.


In [28]:
class UserFactContextProvider(ContextProvider):
    def __init__(self, user_id: str, store: FactStore, max_facts: int = 8) -> None:
        self.user_id = user_id
        self.store = store
        self.max_facts = max_facts
        self.cached_facts: list[str] = []
        self._patterns = [
            re.compile(r"\bmy name is (?P<value>[A-Za-z\s]{2,60})", re.IGNORECASE),
            re.compile(r"\bi live in (?P<value>[A-Za-z\s,]{2,80})", re.IGNORECASE),
            re.compile(r"\bmy favourite (?P<topic>\w+) is (?P<value>[A-Za-z0-9\s,\-']{2,80})", re.IGNORECASE),
            re.compile(r"\bmy favorite (?P<topic>\w+) is (?P<value>[A-Za-z0-9\s,\-']{2,80})", re.IGNORECASE),
            re.compile(r"\bi work as (?P<value>[A-Za-z0-9\s,\-']{2,80})", re.IGNORECASE),
        ]

    async def __aenter__(self) -> "UserFactContextProvider":
        await super().__aenter__()
        self.cached_facts = self.store.load(self.user_id)[-self.max_facts :]
        return self

    async def thread_created(self, thread_id: str | None) -> None:
        self.cached_facts = self.store.load(self.user_id)[-self.max_facts :]

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs) -> Context:
        if not self.cached_facts:
            return Context()
        recent_facts = self.cached_facts[-self.max_facts :]
        bullet_list = "\n".join(f"- {item}" for item in recent_facts)
        instructions = f"User information:\n{bullet_list}\n"
        return Context(instructions=instructions)

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs,
    ) -> None:
        if invoke_exception is not None:
            return
        collected: list[str] = []
        for message in self._ensure_sequence(request_messages):
            if getattr(message.role, "value", "") != "user":
                continue
            collected.extend(self._extract_facts(message.text))
        if not collected:
            return
        self.store.add(self.user_id, collected)
        self.cached_facts = self.store.load(self.user_id)[-self.max_facts :]

    def _ensure_sequence(
        self,
        messages: ChatMessage | Sequence[ChatMessage],
    ) -> list[ChatMessage]:
        if isinstance(messages, Sequence):
            return list(messages)
        return [messages]

    def _extract_facts(self, text: str | None) -> list[str]:
        if not text:
            return []
        facts: list[str] = []
        for pattern in self._patterns:
            match = pattern.search(text)
            if not match:
                continue
            groups = match.groupdict()
            if "topic" in groups:
                topic = groups["topic"].strip()
                value = groups["value"].strip()
                facts.append(f"User's {topic} is {value}")
            else:
                value = groups["value"].strip()
                if "name is" in pattern.pattern:
                    facts.append(f"User name is {value}")
                elif "live in" in pattern.pattern:
                    facts.append(f"User lives in {value}")
                else:
                    facts.append(value)
        marker = "remember that"
        lowered = text.lower()
        idx = lowered.find(marker)
        if idx != -1:
            statement = text[idx + len(marker):].strip(' .!')
            if statement:
                facts.append(statement[0].upper() + statement[1:])
        unique: list[str] = []
        seen: set[str] = set()
        for fact in facts:
            normalised = fact.strip()
            if not normalised:
                continue
            key = normalised.lower()
            if key in seen:
                continue
            seen.add(key)
            unique.append(normalised)
        return unique


## Tone helper context provider

A lightweight provider that keeps responses aligned with a specific brand tone. Because context providers are composable, we can layer stylistic instructions with memory-driven instructions.


In [29]:
class ToneContextProvider(ContextProvider):
    def __init__(self, instructions: str) -> None:
        self._instructions = instructions.strip() + "\n"

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs) -> Context:
        return Context(instructions=self._instructions)

## Compose providers and run an Azure AI agent

We combine the tone helper with the fact collector and interact with an Azure AI Agent backed by Azure AI Foundry. The authentication flow matches the other Azure AI notebooks in this repository, relying on the Azure CLI credential.

**Important**: We use a single `thread` object for all conversation turns. This ensures the agent maintains chat history across turns, allowing for a natural conversation flow.


In [30]:
async def run_demo() -> None:
    user_id = "workshop-user"
    store = FactStore()

    providers = AggregateContextProvider([
        ToneContextProvider(
            instructions=(
                "## Response Style\nUse a professional yet friendly tone. If memories are supplied, weave them naturally into the answer."
            )
        ),
        UserFactContextProvider(user_id=user_id, store=store),
    ])

    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(async_credential=credential),
            instructions="You are an Azure AI travel planner that personalises suggestions.",
            context_providers=providers,
        ) as agent,
    ):
        # Create a single thread for the entire conversation
        thread = agent.get_new_thread()
        
        prompts = [
            "Hi, my name is Alex and I live in Berlin. I love museum hopping on weekends.",
            "Please remember that I am planning a trip to Seattle next month.",
            "I prefer boutique hotels near art districts.",
            "Can you suggest a weekend plan that includes something I would enjoy?",
        ]
        for turn, prompt in enumerate(prompts, start=1):
            print(f"Turn {turn} - user: {prompt}")
            response = await agent.run(prompt, thread=thread)
            text = getattr(response, "text", str(response))
            print(f"Turn {turn} - agent: {text}\n")

    print("Stored facts after the run:")
    for fact in store.load(user_id):
        print(f"- {fact}")


### Run the demo conversation

Execute the async helper to see the context providers capture and reuse memories. 

**Important**: Make sure you are logged in to Azure CLI first by running in your terminal:
```bash
az login --use-device-code
```

If you get authentication errors, try clearing and re-logging:
```bash
az account clear
az login --use-device-code
```

In [31]:
await run_demo()


Turn 1 - user: Hi, my name is Alex and I live in Berlin. I love museum hopping on weekends.


[2025-10-25 10:31:40 - c:\Users\kapildhanger\OneDrive - Microsoft\Microsoft_Kapil\AzureCustomers\Everest\NYC_Workshop\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Turn 1 - agent: Hello Alex! Living in Berlin, you’re surrounded by some of the world's most iconic museums—what an incredible city for someone who loves cultural pursuits like yourself. Let me offer a few great ideas to enrich your museum-hop weekends:  

### Berlin’s Must-Sees for Museum Lovers:
1. **Museum Island (Museumsinsel):** If you haven’t already explored this UNESCO World Heritage Site thoroughly, it’s a must! From the Pergamon Museum to the Alte Nationalgalerie, each offers something unique—from reconstructed ancient temples to breathtaking art collections. Plan to take your time—perhaps dedicating a day to explore one or two museums at depth.

2. **Hamburger Bahnhof:** Dive into the contemporary art scene at this museum housed in a former railway station. It's a favorite for modern and experimental art lovers.

3. **Berlinische Galerie:** Perfect for discovering Berlin’s avant-garde movements. A mix of visual art, photography, and architecture that speaks to Berlin's creati

[2025-10-25 10:31:42 - c:\Users\kapildhanger\OneDrive - Microsoft\Microsoft_Kapil\AzureCustomers\Everest\NYC_Workshop\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Turn 2 - agent: Got it, Alex! I’ll keep your upcoming trip to Seattle in mind while planning suggestions tailored to your love of museums and art. Seattle has a vibrant cultural scene that's sure to delight you. Let me know if you'd like me to start crafting an itinerary for your trip!

Turn 3 - user: I prefer boutique hotels near art districts.


[2025-10-25 10:31:49 - c:\Users\kapildhanger\OneDrive - Microsoft\Microsoft_Kapil\AzureCustomers\Everest\NYC_Workshop\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Turn 3 - agent: Great to know, Alex! Boutique hotels near Seattle’s art districts will perfectly complement your passion for museums. The city has some fantastic options that combine charm, stylish vibes, and convenient locations close to cultural hotspots. Here are some tailored recommendations for your trip:

---

### **Boutique Hotel Suggestions**
1. **Hotel Ändra**  
   - Location: Belltown District  
   - Why You’ll Love It: This chic, Scandinavian-inspired hotel is set right in one of Seattle’s trendiest areas. You’ll be steps away from fine restaurants, quirky galleries, and a vibrant nightlife scene. Plus, it’s a short walk to the Seattle Art Museum (SAM) downtown.  

2. **The Ace Hotel**  
   - Location: Belltown  
   - Why You’ll Love It: A bohemian option with unique, art-filled decor in every room, this boutique gem aligns beautifully with the creative sensibilities of the area. It fits perfectly with your love for art and cultural immersion.  

3. **The Palihotel Seattle**

## Next steps

- Swap the in-memory store for Azure Cosmos DB, Azure Cache for Redis, or another persistent store.
- Use an Azure AI summarisation deployment within `invoked` to create higher level memories.
- Combine additional context providers (for example, retrieval augmented providers) and let `AggregateContextProvider` merge them automatically.
